### Indexing API

In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv('../application/.env'))

True

Lets add Documents and Embeddings!

In [2]:
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter

loader = DirectoryLoader('./restaurant', glob="**/*.txt", loader_cls=TextLoader)
data = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=150, chunk_overlap=20)
docs = text_splitter.split_documents(data)
print(len(docs))

FileNotFoundError: Directory not found: './hotels'

In [ ]:
import os

host = os.getenv("PG_VECTOR_HOST")
user = os.getenv("PG_VECTOR_USER")
password = os.getenv("PG_VECTOR_PASSWORD")
COLLECTION_NAME = os.getenv("PG_COLLECTION_NAME")

CONNECTION_STRING = f"postgresql+psycopg2://{user}:{password}@{host}:5432/{COLLECTION_NAME}"
CONNECTION_STRING

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.pgvector import PGVector

embeddings = OpenAIEmbeddings()

vector_store = PGVector(
    embedding_function=embeddings,
    collection_name=COLLECTION_NAME,
    connection_string=CONNECTION_STRING,
)

In [ ]:
from langchain.indexes import SQLRecordManager, index

In [ ]:
namespace = f"pgvector/{COLLECTION_NAME}"
record_manager = SQLRecordManager(
    namespace, db_url=CONNECTION_STRING
)

In [ ]:
record_manager.create_schema()

Update the documents to see some changes (2nd run)

In [ ]:
index(
    docs,
    record_manager,
    vector_store,
    cleanup=None,
    source_id_key="source",
)

In [ ]:
from langchain.schema import Document

docs[1].page_content = "updated"
del docs[6]
docs.append(Document(page_content="new content", metadata={"source": "important"}))

In [ ]:
index(
    docs,
    record_manager,
    vector_store,
    cleanup=None,
    source_id_key="source",
)

In [ ]:
docs[1].page_content = "updated again"
del docs[2]
del docs[3]
del docs[4]
docs.append(Document(page_content="more new content", metadata={"source": "important"}))

In [ ]:
index(
    docs,
    record_manager,
    vector_store,
    cleanup="incremental",
    source_id_key="source",
)

In [ ]:
index(
    [],
    record_manager,
    vector_store,
    cleanup="incremental",
    source_id_key="source",
)

In [ ]:
index([], record_manager, vector_store, cleanup="full", source_id_key="source")